In [1]:
import os
import glob
import pandas as pd
import re
from tqdm import tqdm
os.chdir("/data/localhost/taufiq/bigears-data")

In [2]:
extension = '.csv'
all_filenames = [i for i in glob.glob('*{}'.format(extension))]
biogears_data = pd.DataFrame()
for f in tqdm(all_filenames):
    if os.path.getsize(f) > 0:
        df = pd.read_csv(f)
        m = re.search('SimulateMIMIC_(.+?)_.csv', f)
        if m:
            icustay_id = m.group(1)
            df['icustay_id'] = icustay_id
        biogears_data = biogears_data.append(df, ignore_index=True)
        
# biogears_data = pd.concat([pd.read_csv(f) for f in all_filenames if os.path.getsize(f) > 0])

100%|██████████| 6476/6476 [21:21<00:00,  5.05it/s]


In [3]:
times = [10.02, 3600.02]
biogears_data = biogears_data[biogears_data['Time(s)'].isin(times)]
biogears_data['icustay_id'] = biogears_data['icustay_id'].astype(int)

In [4]:
icustayids = []
for icustay_id in biogears_data['icustay_id'].unique():
    if (biogears_data['icustay_id'] == icustay_id).sum() == 2:
        icustayids.append(icustay_id)

In [5]:
biogears_data = biogears_data[biogears_data['icustay_id'].isin(icustayids)].reset_index(drop=True)

In [6]:
biogears_data.head()

,Time(s),HeartRate(1/min),MeanArterialPressure(mmHg),SystolicArterialPressure(mmHg),DiastolicArterialPressure(mmHg),CardiacOutput(mL/min),HemoglobinContent(g),CentralVenousPressure(mmHg),Hematocrit,ArterialBloodPH,...,Piperacillin-BloodConcentration,Tazobactam-BloodConcentration,Glucose-BloodConcentration(ug/mL),Sodium-BloodConcentration(ug/mL),Potassium-BloodConcentration(ug/mL),Chloride-BloodConcentration(ug/mL),Calcium-BloodConcentration(ug/mL),Magnesium-BloodConcentration,Albumin-BloodConcentration(ug/mL),icustay_id
0,10.02,62.2313,90.1170,117.505,61.2284,5247.67,469.204,4.29491,0.459228,7.40294,...,-1.$,-1.$,819.333,3243.29,160.821,3777.32,94.5324,-1.$,44957.8,69184
1,3600.02,58.5146,91.3791,122.405,58.9555,5685.91,469.211,4.69330,0.424163,7.44205,...,0,0,738.848,3207.18,157.634,3662.20,91.3362,-1.$,41539.1,69184
2,10.02,86.5803,101.8150,120.811,80.8178,7196.23,1100.620,4.21137,0.460022,7.46220,...,-1.$,-1.$,842.446,3222.66,159.813,3767.98,93.9551,-1.$,44989.0,38346
3,3600.02,84.5227,102.3460,122.684,79.7390,7595.43,1100.630,4.41927,0.442152,7.49379,...,0,0,757.909,3194.33,157.315,3689.29,91.8278,-1.$,43073.5,38346
4,10.02,83.2297,98.1942,121.029,73.2544,7047.01,765.027,4.15863,0.459815,7.40575,...,-1.$,-1.$,854.989,3239.68,160.122,3784.06,94.5441,-1.$,44990.7,16224


In [7]:
column_mappings = {
    'Albumin-BloodConcentration(ug/mL)': 'Albumin',
    'ArterialCarbonDioxidePressure(mmHg)': 'paCO2',
    'ArterialOxygenPressure(mmHg)': 'paO2',
    'Bicarbonate-BloodConcentration(ug/mL)': 'HCO3',
    'ArterialBloodPH': 'Arterial_pH',
    'Calcium-BloodConcentration(ug/mL)': 'Calcium',
    'Chloride-BloodConcentration(ug/mL)': 'Chloride',
    'Creatinine-BloodConcentration(ug/mL)': 'Creatinine',
    'DiastolicArterialPressure(mmHg)': 'DiaBP',
    'Glucose-BloodConcentration(ug/mL)': 'Glucose',
    'Lactate-BloodConcentration(ug/mL)': 'Arterial_lactate',
    'MeanArterialPressure(mmHg)': 'MeanBP',
    'Potassium-BloodConcentration(ug/mL)': 'Potassium',
    'RespirationRate(1/min)': 'RR',
    'SkinTemperature(degC)': 'Temp_C',
    'Sodium-BloodConcentration(ug/mL)': 'Sodium',
    'SystolicArterialPressure(mmHg)': 'SysBP',
    'WhiteBloodCellCount(ct/uL)': 'WBC_count',
    'HeartRate(1/min)': 'HR'
}

biogears_rename = {}

for k, v in column_mappings.items():
    biogears_rename.update({k: f"{v}-biogears"})

biogears_data = biogears_data.rename(columns=biogears_rename)

In [8]:
biogears_data_t1 = biogears_data[biogears_data['Time(s)']==3600.02].reset_index(drop=True)
biogears_data_t1.head()

,Time(s),HR-biogears,MeanBP-biogears,SysBP-biogears,DiaBP-biogears,CardiacOutput(mL/min),HemoglobinContent(g),CentralVenousPressure(mmHg),Hematocrit,Arterial_pH-biogears,...,Piperacillin-BloodConcentration,Tazobactam-BloodConcentration,Glucose-biogears,Sodium-biogears,Potassium-biogears,Chloride-biogears,Calcium-biogears,Magnesium-BloodConcentration,Albumin-biogears,icustay_id
0,3600.02,58.5146,91.3791,122.405,58.9555,5685.91,469.211,4.69330,0.424163,7.44205,...,0,0,738.848,3207.18,157.634,3662.20,91.3362,-1.$,41539.1,69184
1,3600.02,84.5227,102.3460,122.684,79.7390,7595.43,1100.630,4.41927,0.442152,7.49379,...,0,0,757.909,3194.33,157.315,3689.29,91.8278,-1.$,43073.5,38346
2,3600.02,79.9573,99.8777,124.170,73.1236,7377.83,765.030,4.49610,0.435274,7.42603,...,0,0,758.253,3197.68,157.472,3671.64,91.5508,-1.$,42521.9,16224
3,3600.02,64.0127,93.1179,124.699,59.7810,6258.03,780.008,4.84822,0.435786,7.46128,...,0,0,752.873,3207.00,157.832,3694.44,91.9661,-1.$,42365.0,21620
4,3600.02,72.4224,93.2160,126.911,57.7194,7201.44,398.287,4.94234,0.419786,7.45138,...,0,0,756.832,3213.80,157.858,3655.27,91.0412,-1.$,41142.2,1204


In [9]:
len(biogears_data_t1)

5330

In [10]:
MIMICtable = pd.read_csv("/data/localhost/taufiq/export-dir/MIMIC-1hourly-length-2.csv")
MIMICtable['icustay_id'] = MIMICtable['icustay_id'].astype(int)
MIMICtable.head()

,bloc,icustay_id,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,mortality_90d,...,mechvent,Shock_Index,PaO2_FiO2,median_dose_vaso,max_dose_vaso,input_total,input_1hourly,output_total,output_1hourly,cumulated_balance
0,1,65537,7.139723e+09,0.0,22799.404780,4.0,1.0,0.0,0.0,0.0,...,0.0,0.687500,363.888889,0.18,0.18,4020.493550,29.994183,7235.0,3605.0,-3184.512267
1,2,65537,7.139727e+09,0.0,22799.404780,4.0,1.0,0.0,0.0,0.0,...,0.0,0.602564,430.555556,0.00,0.00,4050.487733,515.407667,10760.0,3525.0,-6194.104600
2,1,3,7.245400e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.663900,599.999991,0.00,0.00,6267.000000,0.000000,9490.0,4705.0,-3223.000000
3,2,3,7.245403e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.672340,339.999995,0.00,0.00,6267.000000,10.000000,14095.0,4605.0,-7818.000000
4,1,65544,7.056917e+09,0.0,24000.436400,3.0,0.0,0.0,0.0,0.0,...,0.0,0.770925,938.636364,0.00,0.00,11332.000000,0.000000,8210.0,4105.0,3122.000000


In [11]:
MIMICtable_filtered = MIMICtable[MIMICtable['icustay_id'].isin(biogears_data['icustay_id'])]

In [12]:
MIMICtable_filtered_t0 = MIMICtable_filtered[MIMICtable_filtered['bloc']==1].reset_index()
MIMICtable_filtered_t1 = MIMICtable_filtered[MIMICtable_filtered['bloc']==2][[
       'icustay_id', 'RR', 'HR', 'SysBP', 'MeanBP', 'DiaBP', 'RR',
       'SpO2', 'Temp_C', 'FiO2_1', 'Potassium', 'Sodium', 'Chloride',
       'Glucose', 'BUN', 'Creatinine', 'Magnesium', 'Calcium', 'Ionised_Ca',
       'CO2_mEqL', 'SGOT', 'SGPT', 'Total_bili', 'Albumin', 'Hb', 'WBC_count',
       'Platelets_count', 'PTT', 'PT', 'INR', 'Arterial_pH', 'paO2', 'paCO2',
       'Arterial_BE', 'HCO3', 'Arterial_lactate']].reset_index()

In [13]:
col_dict = {}
for col in MIMICtable_filtered_t1.columns:
    if col != 'icustay_id':
        col_dict.update({col: f"{col}-t1"})

MIMICtable_filtered_t1 = MIMICtable_filtered_t1.rename(columns=col_dict)

In [14]:
data_combined = MIMICtable_filtered_t0.merge(MIMICtable_filtered_t1, left_on='icustay_id', right_on='icustay_id')
data_combined = data_combined.merge(biogears_data_t1, left_on='icustay_id', right_on='icustay_id')

In [15]:
len(data_combined)

5330

In [16]:
probs_data = pd.read_csv("/data/localhost/taufiq/export-dir/probability-data.csv")
probs_data['icustay_id'] = probs_data['icustay_id'].astype(int)

In [17]:
data_combined = data_combined.merge(probs_data, left_on='icustay_id', right_on='icustay_id')

In [18]:
len(data_combined)

1101

In [19]:
inbounds = 1
for col in ['HR', 'SysBP', 'DiaBP', 'MeanBP', 'Sodium', 'Chloride', 'Glucose', 'Creatinine', 'Calcium']:
# for col in ['HR', 'SysBP', 'DiaBP']:
    data_combined[f"{col}_ub"] = data_combined[col]*data_combined['prob'] + (1-data_combined['prob'])*(data_combined[col].max())
    data_combined[f"{col}_lb"] = data_combined[col]*data_combined['prob'] + (1-data_combined['prob'])*(data_combined[col].min())
    inbounds *= data_combined[f'{col}-biogears'].between(data_combined[f"{col}_lb"], data_combined[f"{col}_ub"])
# data_combined['hr_ub'] = data_combined['HR']*data_combined['prob'] + (1-data_combined['prob'])*(data_combined['HR'].max())
# data_combined['hr_lb'] = data_combined['HR']*data_combined['prob'] + (1-data_combined['prob'])*(data_combined['HR'].min())
# data_combined['sysbp_ub'] = data_combined['SysBP']*data_combined['prob'] + (1-data_combined['prob'])*(data_combined['SysBP'].max())
# data_combined['sysbp_lb'] = data_combined['SysBP']*data_combined['prob'] + (1-data_combined['prob'])*(data_combined['SysBP'].min())


In [20]:
inbounds.sum()/len(data_combined)

0.5022706630336058